In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16507139
paper_name = 'narayanaswamy_marcotte_2006' 

In [86]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [87]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [89]:
original_data = pd.read_excel('raw_data/gb-2006-7-1-r6-s2.xlsx',sheet_name='Cellma Grades', skiprows=39)

In [90]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4847 x 61


In [91]:
original_data.head()

,GENE,22_SLIGHT_SHMOOS,24_SLIGHT_SHMOOS,22_NORMAL_SHMOOS,24_NORMAL_SHMOOS,22_OTHER_SHMOOS,24_OTHER_SHMOOS,22_LARGE_INT,24_LARGE_INT,22_LARGE_PEN,...,Large,Small,Round,Pointed,Elongated,Pseudohyphal-like,Clumped,Budding,Polarized Bud Growth,Other
0,YAL002W,1.0,1,0.0,0,0.0,0,0.0,1.0,0.0,...,2,0,0,0,0,0,0,2,0,0
1,YAL004W,3.0,1,1.0,1,0.0,0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
2,YAL005C,3.0,2,1.0,1,0.0,0,0.0,0.0,0.0,...,0,0,0,0,3,0,0,0,0,0
3,YAL007C,3.0,1,1.0,1,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,YAL008W,2.0,2,2.0,2,0.0,0,0.0,0.0,0.0,...,0,3,0,0,3,0,0,0,0,0


In [92]:
original_data['orf'] = original_data['GENE'].astype(str)

In [93]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [94]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [95]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [96]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [GENE, 22_SLIGHT_SHMOOS, 24_SLIGHT_SHMOOS, 22_NORMAL_SHMOOS, 24_NORMAL_SHMOOS, 22_OTHER_SHMOOS, 24_OTHER_SHMOOS, 22_LARGE_INT, 24_LARGE_INT, 22_LARGE_PEN, 24_LARGE_PEN, 22_SMALL_INT, 24_SMALL_INT, 22_SMALL_PEN, 24_SMALL_PEN, 22_ROUND_INT, 24_ROUND_INT, 22_ROUND_PEN, 24_ROUND_PEN, 22_POINTED_INT, 24_POINTED_INT, 22_POINTED_PEN, 24_POINTED_PEN, 22_ELONGATED_INT, 24_ELONGATED_INT, 22_ELONGATED_PEN, 24_ELONGATED_PEN, 22_PSEUDOHYPHAL_INT, 24_PSEUDOHYPHAL_INT, 22_PSEUDOHYPHAL_PEN, 24_PSEUDOHYPHAL_PEN, 22_CLUMPY_INT, 24_CLUMPY_INT, 22_CLUMPY_PEN, 24_CLUMPY_PEN, 22_BUDDING_INT, 24_BUDDING_INT, 22_BUDDING_PEN, 24_BUDDING_PEN, 22_POLARIZEDBUDGROWTH_INT, 24_POLARIZEDBUDGROWTH_INT, 22_POLARIZEDBUDGROWTH_PEN, 24_POLARIZEDBUDGROWTH_PEN, 22_OTHER_INT, 24_OTHER_INT, 22_OTHER_PEN, 24_OTHER_PEN, 22_SHMOO_PROBLEMS, 24_SHMOO_PROBLEMS, 22_STANDARD_PROBLEMS, 24_STANDARD_PROBLEMS, Large, Small, Round, Pointed, Elongated, Pseudohyphal-like, Clumped, Budding, Polarized Bud Growth, Othe

In [97]:
original_data.set_index('orf', inplace=True)
original_data.drop(columns=['GENE'], inplace=True)

In [98]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [99]:
original_data = original_data.groupby(original_data.index).mean()

In [100]:
original_data.shape

(4760, 60)

In [101]:
# Multiply intensity by penetrance
int_cols = [c for c in original_data.columns.values if '_INT' in c]
pen_cols = [c for c in original_data.columns.values if '_PEN' in c]

In [102]:
for ic,c in enumerate(int_cols):
    original_data[int_cols[ic]] = original_data[int_cols[ic]] * original_data[pen_cols[ic]] / 4

In [103]:
original_data.drop(columns=pen_cols, inplace=True)

In [104]:
# Average between observers
obs1_cols = [c for c in original_data.columns.values if '22_' in c]
obs2_cols = [c for c in original_data.columns.values if '24_' in c]

In [105]:
for ic, c in enumerate(obs1_cols):
    original_data[obs1_cols[ic]] = (original_data[obs1_cols[ic]] + original_data[obs2_cols[ic]]) / 2

In [106]:
original_data.drop(columns=obs2_cols, inplace=True)

# Load dataset_ids

In [107]:
mp = pd.read_excel('raw_data/phenotype_mapping.xlsx', sheet_name='Sheet1', skiprows=1)

In [108]:
mp.set_index('Unnamed: 0', inplace=True)

In [111]:
mp.columns = [int(c) for c in mp.columns]

In [112]:
original_data = original_data.reindex(columns=mp.index.values)

In [113]:
original_data.head()

,22_SLIGHT_SHMOOS,22_LARGE_INT,22_SMALL_INT,22_ROUND_INT,22_POINTED_INT,22_ELONGATED_INT,22_PSEUDOHYPHAL_INT,22_CLUMPY_INT,22_BUDDING_INT,22_POLARIZEDBUDGROWTH_INT
orf,,,,,,,,,,
YAL002W,1.0,0.125,0.00,0.0,0.0,0.00,0.0,0.0,0.125,0.0
YAL004W,2.0,0.000,0.00,0.0,0.0,0.00,0.0,0.0,0.000,0.0
YAL005C,2.5,0.000,0.00,0.0,0.0,0.25,0.0,0.0,0.000,0.0
YAL007C,2.0,0.000,0.00,0.0,0.0,0.00,0.0,0.0,0.000,0.0
YAL008W,2.0,0.000,0.25,0.0,0.0,0.25,0.0,0.0,0.000,0.0


In [114]:
original_data2 = pd.DataFrame(index=original_data.index, columns=mp.columns, data=0)

In [115]:
for p in mp.index.values:
    for d in mp.columns.values:
        v = mp.loc[p,d]
        if np.abs(v) > 0:
            original_data2.loc[:,d] = original_data2.loc[:,d] + original_data.loc[:,p] * v

In [116]:
original_data2.head()

,593,595,596,597,598,600,601,599,548
orf,,,,,,,,,
YAL002W,0.125,0.0,0.0,0.00,0.0,0.125,0.0,0.0,-1.0
YAL004W,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0
YAL005C,0.000,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.5
YAL007C,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0
YAL008W,-0.250,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.0


# Prepare the final dataset

In [117]:
data = original_data2.copy()

In [118]:
dataset_ids = data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [120]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [121]:
data.head()

dataset_id,593,595,596,597,598,600,601,599,548
data_type,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,
YAL002W,0.125,0.0,0.0,0.00,0.0,0.125,0.0,0.0,-1.0
YAL004W,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0
YAL005C,0.000,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.5
YAL007C,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0
YAL008W,-0.250,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.0


## Subset to the genes currently in SGD

In [122]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [123]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,593,595,596,597,598,600,601,599,548
,data_type,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,
2,YAL002W,0.125,0.0,0.0,0.00,0.0,0.125,0.0,0.0,-1.0
1863,YAL004W,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0
4,YAL005C,0.000,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.5
5,YAL007C,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0
6,YAL008W,-0.250,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.0


# Normalize

In [124]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [125]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [126]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,593,595,596,597,598,600,601,599,548,593,595,596,597,598,600,601,599,548
,data_type,value,value,value,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,
2,YAL002W,0.125,0.0,0.0,0.00,0.0,0.125,0.0,0.0,-1.0,0.336786,0.0,0.0,0.000000,0.0,2.176337,0.0,0.0,0.887436
1863,YAL004W,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0,-0.033679,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.018298
4,YAL005C,0.000,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.5,-0.033679,0.0,0.0,0.837238,0.0,0.000000,0.0,0.0,-0.471164
5,YAL007C,0.000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,-2.0,-0.033679,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.018298
6,YAL008W,-0.250,0.0,0.0,0.25,0.0,0.000,0.0,0.0,-2.0,-0.774608,0.0,0.0,0.837238,0.0,0.000000,0.0,0.0,-0.018298


# Print out

In [127]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [128]:
from IO.save_data_to_db3 import *

In [129]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/9 [00:00<?, ?it/s]

Deleting all datasets for PMID 16507139...
Inserting the new data...


100%|██████████| 9/9 [01:01<00:00,  6.78s/it]

Updating the data_modified_on field...
